# Week 4 - Quality control for sequencing data

Say you had a biological question of interest that could be solved by sequencing. Say you received some funding to perform a sequencing run (lucky you!). The biologist runs the sample in a sequencer (you may know a few machines by now) and deliver a file containing all your reads. It is possible that there were problems in the sequencer or the starting material used for sequencing (DNA). Before you make your awesome discovery and report it, you want to be sure that the data you have is error free. This is where quality control helps (QC).

This week you will write your own modules to perform QC.

## Task 1 - Reading in the readsets

We read in a sample read set. This is created by selecting some reads from the ERR024571 read set.

In [ ]:
import skbio

fname = 'data/reads.fastq' 
registry = skbio.io.read(fname, format = 'fastq', phred_offset = 33)
readset = list(registry)
len(readset)

## Task 2 - Compute *per sequence* qualities

The per sequence quality is the average of base qualities forming the sequence. For sequence 1, that is:

In [ ]:
r1_meta = readset[0].positional_metadata
r1_meta.head()

### Crash course on `pandas` and `numpy`

This is a pandas data frame object. Most data can be stored in data frames and many functions to process them are available in the `pandas` library. We will first try out a few indexing operations and write up a function to compute the per sequence quality.

> Get started with `pandas` with this tutorial - http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

In [ ]:
import pandas as pd
import numpy as np

# all of these indexes give the same results. Think of 2-D matrices
# here we index using positions
print(r1_meta.iloc[:3,])
print(r1_meta.iloc[0:3, 0:1])
print(r1_meta.iloc[0:3, :])
print(r1_meta.iloc[0:3,])
print(r1_meta.iloc[:3,])

Above we use information on the position along the data frame (i.e. rows 0:3 and column 0:1). Instead of this, we could use *row names* and *column names* to index. In the example below, 0:2 refers to the rows with names '0', '1', and '2' and the column with the name 'quality'. Since we slice the columns and select just one, what we get is a `numpy` array.

A simple way to reason about these `numpy` arrays is to think of them as vectors and matrices. A vector or matrix with named rows and columns is a `pandas` data frame.

In [ ]:
#this gives the same result but returns an array (think vectors)
# here we index using names - 0, 1, and 2 are row names rather than positions
print(r1_meta.loc[0:2, 'quality'])

In [ ]:
# retrieve base quality at position 6
r1_qualities = r1_meta.loc[:, 'quality']
r1_qualities[2]

We can now write a function to compute the sequence quality of a given sequence

In [ ]:
def read_quality_with_loop(read):
    """
    Compute the quality of a given read (DNA sequence).
    This is the average of base qualities.
    Return a single number representing the quality
    """
    
    #retrieve metadata
    metadata = read.positional_metadata
    #select the column containing quality information
    qualities = metadata.loc[:, 'quality']
    
    avg_quality = 0
    for phred_score in qualities:
        avg_quality += phred_score
    return avg_quality / len(qualities)
    

In [ ]:
print(read_quality_with_loop(readset[0])) # should give 27.36

The same can be done in an easier way using functions from the `numpy` library. This implementation is also considerably faster.

In [ ]:
def read_quality(read):
    """
    Compute the quality of a given read (DNA sequence).
    This is the average of base qualities.
    Return a single number representing the quality
    """
    
    return np.mean(read.positional_metadata.loc[:, 'quality'])


In [ ]:
print(read_quality_with_loop(readset[0]))

## Task 3 - Compute *per sequence* quality distribution

We compute read qualities for all sequences and compute a sequence quality histogram. This is done by binning the data. We will create bins of integers i.e. given an integer $i$ the interval is $[i, i + 1)$, that is all numbers between $i$ and $i + 1$ including $i$ but excluding $i + 1$.

We will create a dictionary using this. Keys in the dictionary will be $i$ and values will be the number of quality scores falling in the interval $[i, i + 1)$.

The `np.floor` function can be used to round down quality scores to the nearest integer.

In [ ]:
print(np.floor(30.01))
print(np.floor(30.99))

In [ ]:
def read_histogram_dict(reads):
    """
    Compute a dictionary representing the histogram
    Keys are the starting point of the interval
    Values are the number of items in the interval
    """
    
    # compute read qualities for sequences
    read_qualities = [read_quality(r) for r in reads]
    # compute keys for each read
    distribution_keys = [np.floor(r) for r in read_qualities]
    
    #build dictionary
    hist_dict = {}
    for k in distribution_keys:
        if k in hist_dict.keys():
            hist_dict[k] += 1
        else:
            hist_dict[k] = 1 #initialise key
    return hist_dict


In [ ]:
# alternate implementation using the collections.Counter function
#     which counts the occurence of items in a list

import collections

def read_histogram_dict(reads):
    """
    Compute a dictionary representing the histogram
    Keys are the starting point of the interval
    Values are the number of items in the interval
    """
    
    #get keys
    distribution_keys = [np.floor(read_quality(r)) for r in reads]
    return dict(collections.Counter(distribution_keys))

In [ ]:
read_histogram_dict(readset[0:4]) # should give {27.0: 2, 28.0: 1, 26.0: 1}

Lets get to plotting!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

hist_data = read_histogram_dict(readset)
plt.scatter(hist_data.keys(), hist_data.values())
plt.xlabel('Phred quality score')
plt.ylabel('Frequency')
plt.show()

## Task 4 - Use FastQC to perform the analysis

The above analysis and more can be performed using the FastQC program. This is a Java implementation therefore is considerably faster. Use this to analyse the full readset. You can invoke terminal commands from a jupyter notebook by preceeding it with an exclamation mark as in the cell below (you can just type the command in a terminal directly).

Note: fastqc can also be installed using conda using the command `conda install -c bioconda fastqc`. You can access the manual by typing `man fastqc` or `!man fastqc` in a jupyter notebook. The `man` command can be used to access manual pages for many linux commands (try `man ls`, press *q* to exit).

In [ ]:
!fastqc -o ./ data/reads.fastq

In [ ]:
#!fastqc -o ./ data/ERR1817984.fastq.gz

The report can be viewed by right clicking the file (.html) and selecting *Open in New Browser Tab*

## Task 5 - What does this all mean?

The report shows some metrics where the data fails QC checks. Try answering some of these questions:

1. Is this a good readset to work with? What could the downstream effect be, say, if it was to be used to perform SNP calling?
2. How do quality scores vary across positions? Do the chemistries of sequencing assays explain this?
3. How should the sequence composition be across positions? Would this pattern be the same across species?

You should refer to this documentation to help you figure out some of these questions

> Are you convinced of the importance of Quality Control in sequence analysis?

This is but one layer of quality control. There are many more layers of QC depending on the task at hand. For instance, if RNA-seq was performed for the purpose of differential expression analysis, further checks and filters would need to be applied such as filtering out of genes with low read counts. QC should form an important component of all analyses and appropriate time should be invested in it. This will prevent false discoveries from being made and save a lot of people a lot of effort.

When dealing with multiple readsets, MultiQC (python based!) provides a summary of all FastQC reports. See https://multiqc.info/ and play around with the example on this page. It has amazing 